# parse tbl out from HMM hits output

In [ ]:
HMM_SEARCH_OUT = "/Users/daffa/Documents/Work/jspp67_bioinf/results/hmm_homology/prpF_KT2440_magnoliopsida/prpF_KT2440_magnoliopsida-06-results.tbl"

# kegg scraping (pre LLM-planning)

In [6]:
import pandas as pd
from pathlib import Path

In [2]:
string = """
8071409
K01649 2-isopropylmalate synthase [EC:2.3.3.13] | (RefSeq) 2-isopropylmalate synthase A
8057865
K01641 hydroxymethylglutaryl-CoA synthase [EC:2.3.3.10] | (RefSeq) hydroxymethylglutaryl-CoA synthase isoform X1
8085635
K01649 2-isopropylmalate synthase [EC:2.3.3.13] | (RefSeq) 2-isopropylmalate synthase A
8073811
K01647 citrate synthase [EC:2.3.3.1] | (RefSeq) citrate synthase, glyoxysomal isoform X1
8077981
K01641 hydroxymethylglutaryl-CoA synthase [EC:2.3.3.10] | (RefSeq) hydroxymethylglutaryl-CoA synthase
110433071
K01641 hydroxymethylglutaryl-CoA synthase [EC:2.3.3.10] | (RefSeq) hydroxymethylglutaryl-CoA synthase-like
8071625
K01647 citrate synthase [EC:2.3.3.1] | (RefSeq) citrate synthase 4, mitochondrial
"""


In [3]:
def kegg_str_to_df(string):
    lines = [line for line in string.split('\n') if line.strip()]
    list_gene = [lines[i] for i in range(0, len(lines), 2)]
    list_annot = [lines[i] for i in range(1, len(lines), 2)]
    d = {"geneID": list_gene, "annotation": list_annot}    
    return pd.DataFrame(data=d)


In [4]:
kegg_str_to_df(string)

,geneID,annotation
0,8071409,K01649 2-isopropylmalate synthase [EC:2.3.3.13...
1,8057865,K01641 hydroxymethylglutaryl-CoA synthase [EC:...
2,8085635,K01649 2-isopropylmalate synthase [EC:2.3.3.13...
3,8073811,K01647 citrate synthase [EC:2.3.3.1] | (RefSeq...
4,8077981,K01641 hydroxymethylglutaryl-CoA synthase [EC:...
5,110433071,K01641 hydroxymethylglutaryl-CoA synthase [EC:...
6,8071625,K01647 citrate synthase [EC:2.3.3.1] | (RefSeq...


In [ ]:
KEGG_DIR = 'data/reference/KEGG'


## Extract Sorghum genes from KEGG pathway map00660
Parse KEGG FTP `.list` files to get all *Sorghum bicolor* genes in pathway **map00660** (C5-Branched dibasic acid metabolism), enriched with KO, NCBI Gene ID, and NCBI Protein ID.

In [7]:
def parse_kegg_list(filepath, col_names=('source', 'target')):
    """Parse a KEGG two-column tab-separated .list file into a DataFrame.
    Strips namespace prefixes (e.g., 'sbi:', 'ko:', 'path:')."""
    df = pd.read_csv(filepath, sep='\t', header=None, names=col_names)
    for col in df.columns:
        df[col] = df[col].str.split(':', n=1).str[1]
    return df


In [8]:
PATHWAY = 'map00660'
ORGANISM = 'sbi'
KEGG_DIR = Path('data/reference/KEGG')

# 1. Get sorghum genes in pathway sbi00660
sbi_pathway = parse_kegg_list(KEGG_DIR / ORGANISM / f'{ORGANISM}_pathway.list', ('gene', 'pathway'))
pathway_id = PATHWAY.replace('map', ORGANISM)  # map00660 -> sbi00660
genes_in_pathway = sbi_pathway[sbi_pathway['pathway'] == pathway_id][['gene']].copy()
print(f"Found {len(genes_in_pathway)} {ORGANISM} genes in pathway {pathway_id}")
genes_in_pathway


Found 6 sbi genes in pathway sbi00660


,gene
3407,8055065
3408,8055458
3409,8056085
3410,8056510
3411,8073966
3412,8076795


In [9]:
# 2. Enrich with KO ortholog assignments
sbi_ko = parse_kegg_list(KEGG_DIR / ORGANISM / f'{ORGANISM}_ko.list', ('gene', 'ko'))
genes_in_pathway = genes_in_pathway.merge(sbi_ko, on='gene', how='left')

# 3. Enrich with NCBI Gene IDs
sbi_geneid = parse_kegg_list(KEGG_DIR / ORGANISM / f'{ORGANISM}_ncbi-geneid.list', ('gene', 'ncbi_geneid'))
genes_in_pathway = genes_in_pathway.merge(sbi_geneid, on='gene', how='left')

# 4. Enrich with NCBI Protein accessions
sbi_protid = parse_kegg_list(KEGG_DIR / ORGANISM / f'{ORGANISM}_ncbi-proteinid.list', ('gene', 'ncbi_proteinid'))
genes_in_pathway = genes_in_pathway.merge(sbi_protid, on='gene', how='left')

print(f"Final table: {len(genes_in_pathway)} rows x {len(genes_in_pathway.columns)} columns")
genes_in_pathway


Final table: 6 rows x 4 columns


,gene,ko,ncbi_geneid,ncbi_proteinid
0,8055065,K01703,8055065,XP_002453237
1,8055458,K00052,8055458,XP_002447363
2,8056085,K01652,8056085,XP_021315526
3,8056510,K01653,8056510,XP_002452423
4,8073966,K01704,8073966,XP_002452816
5,8076795,K01653,8076795,XP_002449227


In [10]:
# 5. Save to TSV
output_dir = Path('results/kegg')
output_dir.mkdir(parents=True, exist_ok=True)
output_path = output_dir / f'{ORGANISM}_{PATHWAY}_genes.tsv'
genes_in_pathway.to_csv(output_path, sep='\t', index=False)
print(f"Saved to {output_path}")


Saved to results/kegg/sbi_map00660_genes.tsv


In [22]:
def parse_input_file(file):
    """
    read pathway gene file
    output df that contains sp:gene -delimiter- accessionID
    """
    with open(file, 'r') as path_gene_f:
        lines = path_gene_f.readlines()
        lines.remove('species\tgene\tko\tncbi_geneid\tncbi_proteinid\n') # remove header the hard way
        sp_gene_col = []
        acc_col =[]
        for line in lines:
            # split tabs
            sp = line.split('\t')[0]
            geneid = line.split('\t')[1]
            kegg_ortho = line.split('\t')[2]
            prot_acc = line.split('\t')[3]
            # assign to prev defined list, then eventually dataframe
            sp_gene_col.append(':'.join((sp, geneid)))
            acc_col.append(prot_acc)

        d = {"gene": sp_gene_col, "accession":acc_col}
        df = pd.DataFrame(data=d)
    return df

In [23]:
sbi_map00660 = parse_input_file('/Users/daffa/Documents/Work/jspp67_bioinf/results/kegg/sbi_map00660_genes.tsv')

In [24]:
sbi_map00660

,gene,accession
0,sbi:8055065,8055065
1,sbi:8055458,8055458
2,sbi:8056085,8056085
3,sbi:8056510,8056510
4,sbi:8073966,8073966
5,sbi:8076795,8076795
